In [1]:
from urllib.parse import urlparse# %%
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
import time
from bs4 import BeautifulSoup
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import csv
import os
import requests
from selenium.webdriver.support.ui import Select
from datetime import datetime
import os
from selenium.webdriver.support.ui import Select
import csv
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
import openpyxl
from collections import Counter
import re
import pandas as pd
from selenium.common.exceptions import TimeoutException
import json
from urllib.parse import quote
from lxml import etree
import json
# from openai import OpenAI
import json
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
# import cloudscraper
# from bs4 import BeautifulSoup
from lxml import html as html_parser
import requests
import os

# loading data from .env file
from dotenv import load_dotenv
import pyautogui

# import data combinations
# from term_whole_data import *
from whole_life_data import *
# from endowment_products_data import *

def get_driver():
    options = Options()
    # Enable headless mode  
    # options.add_argument("--headless=new")
    # options.add_argument("--disable-gpu")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # open specific chrome proofile
    profile_path=os.path.join(os.getcwd(),"profile")
    options.add_argument(f"user-data-dir={profile_path}")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"--log-level=3")
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=options)
    # driver = webdriver.Chrome(options=options)
    action = ActionChains(driver)
    return driver, action


def switch_to_window(url_part,driver):
    for handle in driver.window_handles:
        driver.switch_to.window(handle)
        if url_part in driver.current_url:
            break

driver,action= get_driver()

In [2]:
data = []

class WholeifeAutomation:
    def __init__(self, search_data):
        self.search_data = search_data

    def do_search(self, max_retries=1000):
        print("Setting search options for Whole Life products")
        for attempt in range(max_retries):
            try:
                self.search_whole_life()
                print("Searching is done...")
                return
            except AttributeError as e:
                print(f"Attempt {attempt+1} failed with error: {e}")
                if attempt == max_retries - 1:
                    raise
                print("Retrying search_whole_life...")
            except Exception as e:
                print(f"Unexpected error: {e}")
                raise

    def filter_specific_providers(self):        
        # select only specific providers
        providers_selectors = [
            '[title="China Taiping Insurance (Singapore) Pte. Ltd."]',
            '[title="Manulife (Singapore) Pte. Ltd."]',
            '[title="Tokio Marine Life Insurance Singapore Ltd"]',
            '[title="China Life Insurance (Singapore) Pte. Ltd."]',
            '[title="Income Insurance Limited"]',
            '[title="Singapore Life Ltd."]'
        ]

        # first unchecking the box
        uncheck_all = self.wait_element(By.CSS_SELECTOR, '[id="AllInsurersCHK"]', 20)
        uncheck_all.click()

        self.wait_element(By.CSS_SELECTOR, '[class="search-head"]', 20).click()

        for provider_selector in providers_selectors:
            ele = self.wait_element(using=By.CSS_SELECTOR, css_code=provider_selector, timeout=2)        
            if ele:
                checkbox = ele.find_element(By.XPATH, "..//input[@type='checkbox']")
                checkbox.click()

    def search_whole_life(self):
        self.get_driver_with_retry()
        # clicking on specific products type section
        time.sleep(0.2)
        element = self.wait_element(By.XPATH, '//label[contains(text(),"Whole Life products")]', 20)
        if element is None:
            raise AttributeError("Whole Life products label not found")
        element.click()
        
        element = self.wait_element(By.CSS_SELECTOR, '[data-id="non-bips"]', 20)
        if element is None:
            raise AttributeError("non-bips element not found")
        element.click()

        # filling the date box
        dob_box = self.wait_element(By.CSS_SELECTOR, '[name="dob"]', 20)
        if dob_box is None:
            raise AttributeError("DOB box not found")
        dob_box.clear()
        # calculating the dob based on age
        dob = datetime.now().year - self.search_data["age"]
        dob_box.send_keys(f"01/01/{dob}")

        # filling gender
        element = self.wait_element(By.CSS_SELECTOR, f'[data-id="{self.search_data["gender"]}"]', 20)
        if element is None:
            raise AttributeError("Gender element not found")
        element.click()

        # clicking smoker `yes`
        element = self.wait_element(By.CSS_SELECTOR, f'[data-id="{self.search_data["smoker"]}"]', 20)
        if element is None:
            raise AttributeError("Smoker element not found")
        element.click()

        # selecting premium term
        premium_term_term_ele = self.wait_element(By.CSS_SELECTOR, '.premiumTermNonDcips a.select2-choice', 20)
        if premium_term_term_ele is None:
            raise AttributeError("Premium term element not found")
        premium_term_term_ele.click()
        time.sleep(0.2)
        print("coverage term text: ", self.search_data)
        premium_term_text = self.search_data["premium_term"]

        option = self.wait_element(By.XPATH, f'//div[contains(text(),"{premium_term_text}")]', 20)
        if option is None:
            raise AttributeError("Premium term option not found")
        option.click()

        # selecting sum assured
        sum_assured_text = self.search_data["sum_assured"]
        sum_assured_ele = self.wait_element(By.CSS_SELECTOR, '.SANonDCIPWLSingle a.select2-choice', 20)
        if sum_assured_ele is None:
            raise AttributeError("Sum assured element not found")
        sum_assured_ele.click()
        time.sleep(0.2)
        option = self.wait_element(By.XPATH, f'//div[contains(text(),"{sum_assured_text}")]', 20)
        if option is None:
            raise AttributeError("Sum assured option not found")
        option.click()

        time.sleep(0.2)
        # clicking on critical illness button
        element = self.wait_element(By.CSS_SELECTOR, f'#cretical-ill [data-id="{self.search_data["critical_illness"]}"]', 20)
        if element is None:
            raise AttributeError("Critical illness element not found")
        element.click()

        # clicking on sort A-Z button
        sorting_ele = self.wait_element(By.CSS_SELECTOR, '.sortWLGroup a.select2-choice', 20)
        if sorting_ele is None:
            raise AttributeError("Sort element not found")
        sorting_ele.click()
        time.sleep(0.2)
        option = self.wait_element(By.XPATH, '//div[contains(text(),"Insurer (A - Z)")]', 20)
        if option is None:
            raise AttributeError("Sort option not found")
        option.click()

        # clicking on search button
        element = self.wait_element(By.CSS_SELECTOR, '.btn-search', 20)
        if element is None:
            raise AttributeError("Search button not found")
        element.click()

        # clicking on `i agree`
        element = self.wait_element(By.CSS_SELECTOR, '[id="iUnderstant"]', 20)
        if element is None:
            raise AttributeError("I understand button not found")
        element.click()
    
    def scroll_container(self):
        products_container = self.wait_element(By.CSS_SELECTOR, '#result_container', 20)
        
        # Initial scroll position
        last_height = driver.execute_script("return arguments[0].scrollHeight", products_container)
        scroll_attempts = 0
        max_attempts = 10  # Limit tries to prevent infinite loops

        while scroll_attempts < max_attempts:
            # Scroll down to bottom
            scroll_script = "arguments[0].scrollTo(0, arguments[0].scrollHeight);"
            driver.execute_script(scroll_script, products_container)
            
            # Wait to load page
            time.sleep(0.2)
            
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return arguments[0].scrollHeight", products_container)
            
            # If heights are the same, we've reached the bottom or no new content is loading
            if new_height == last_height:
                break
                
            last_height = new_height
            scroll_attempts += 1

        print(f"Finished scrolling after {scroll_attempts} attempts")

    def check_if_provider_is_required(self, provider_name):
        # print("checking provider: ", provider_name)
        all_providers = [
            "China Life Insurance (Singapore) Pte. Ltd.",
            "China Taiping Insurance (Singapore) Pte. Ltd.",
            "Income Insurance Limited",
            "Manulife (Singapore) Pte. Ltd.",
            "Singapore Life Ltd.",
            "Tokio Marine Life Insurance Singapore Ltd"
        ]
        
        return provider_name in all_providers
    
    def get_driver_with_retry(self, retries=1000, delay=2):
        for attempt in range(retries):
            try:
                driver.get("https://www.comparefirst.sg/wap/homeEvent.action")
                time.sleep(delay)
                self.wait_element(By.XPATH, '//label[contains(text(),"Whole Life products")]', 20)
                return True
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                time.sleep(delay)
        raise Exception("Failed to load page after several attempts.")

    def scrape_products(self):
        global data

        try:
            # getting total products available to scrape it looks like `showing 5 of 25`
            total_products_text = self.wait_element(By.CSS_SELECTOR, '[id="lblInsurerChkAll"]', 20).text
            print("Total products text: ", total_products_text)
            match = re.search(r"\((\d+)\)", total_products_text)
            total_products = int(match.group(1)) if match else 0
        except:
            return None

        index = 0

        while index < total_products:
            print(f"scraping data for product {index + 1} of {total_products}")
            # scroll products box to load all products
            self.scroll_container()

            # getting all `view details` buttons
            try:
                providers = self.wait_element(By.CSS_SELECTOR, '.result_content_inner h3', 20, wait_all=True)
                products = self.wait_element(By.CSS_SELECTOR, '[id="sProdName"]', 20, wait_all=True)
                provider = providers[index].get_attribute('textContent')
                product = products[index].get_attribute('textContent')

                print("checking provider: ", provider)
                print("checking product: ", product)

                if not self.check_if_provider_is_required(provider):
                    print(f"Skipping product {index + 1} - Provider: {provider}, Product: {product}")
                    index += 1
                    continue

                # 🟡 Find the view details button again right before clicking
                view_details_buttons = self.wait_element(By.CSS_SELECTOR, '[class="search_detail"]', 20, wait_all=True)
                self.js_click(view_details_buttons[index])

            except Exception as e:
                print(f"Exception in loading product {index + 1}: {e}")
                index += 1
                continue

            # extracting data
            try:
                provider = self.wait_element(By.CSS_SELECTOR, '.product_details_right h3', 20).text
            except:
                print(f"Failed to extract provider details for product {index + 1}")
                driver.back()
                time.sleep(1)
                index += 1
                continue

            # Skip if provider is not required (double-check)
            if not self.check_if_provider_is_required(provider):
                print(f"Skipping product {index + 1} - Provider: {provider}")
                driver.back()
                time.sleep(1)
                index += 1
                continue

            product = self.wait_element(By.CSS_SELECTOR, '.prod-desc', 20).text
            print(f"scraping data for {product} product")
            
            coverage_term = self.wait_element(By.CSS_SELECTOR, '.policy-term', 20, wait_all=True)[0].text
            subtitle = self.wait_element(By.CSS_SELECTOR, '.Subtitle', 20, wait_all=True)[0].text


            elements = self.wait_element(By.CSS_SELECTOR, '.pay-period', 20, wait_all=True)
            premium_term = ""
            for el in elements:
                text = el.text.strip()
                if text:
                    premium_term = text
                    break
            elements = self.wait_element(By.CSS_SELECTOR, '.annual-premium',20, wait_all=True)

            main_plan_annual_premium = ""
            for el in elements:
                text = el.text.strip()
                if text:
                    main_plan_annual_premium = text
                    break
            elements = self.wait_element(By.CSS_SELECTOR, '.tot-premium',20, wait_all=True)

            tot_premium = ""
            for el in elements:
                text = el.text.strip()
                if text:
                    print("tot_premium text: ", text)
                    tot_premium = self.parse_currency(text)
                    break

            ##Click on the "Death Benefit" tab
            death_tab = self.wait_element(By.CSS_SELECTOR, 'ul.wl_tabs li a[href="#wl_tab1"]', timeout=10)
            if death_tab:
                death_tab.click()
            else:
                print("Death Benefit tab not found")
                
            tab_content = self.wait_element(By.CSS_SELECTOR, '#wl_tab1', timeout=10)
            if not tab_content:
                print("Death Benefit tab content not found")
            else:
                #Get data from tables in tab
                db_years = [el.text for el in tab_content.find_elements(By.CSS_SELECTOR, ".tab-content-table3")]
                db_guaranteed = [el.text for el in tab_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1.tab-content-table4")]
                db_lower_rate = [el.text for el in tab_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1[class*='lr']")]
                db_higher_rate = [el.text for el in tab_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1[class*='hr']")]

            # Click tab "Surrender Value"
            tab2 = self.wait_element(By.CSS_SELECTOR, 'ul.wl_tabs li a[href="#wl_tab2"]', timeout=10)
            if tab2:
                tab2.click()
            else:
                print("Surrender Value tab not found")
            
            # Wait for tab2 content to display
            tab2_content = self.wait_element(By.CSS_SELECTOR, '#wl_tab2', timeout=10)
            if not tab2_content:
                print("Tab2 content not found")
                return
            # Get data in tab2
            sv_years = [el.text for el in tab2_content.find_elements(By.CSS_SELECTOR, ".tab-content-table3")]
            sv_guaranteed = [el.text for el in tab2_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1.tab-content-table4")]
            sv_lower_rate = [el.text for el in tab2_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1[class*='lr']")]
            sv_higher_rate = [el.text for el in tab2_content.find_elements(By.CSS_SELECTOR, ".tab-content-table1[class*='hr']")]


                        
            product_rows = []
            num_years = min(len(db_years), len(db_guaranteed), len(db_lower_rate), len(db_higher_rate),
                len(sv_years), len(sv_guaranteed), len(sv_lower_rate), len(sv_higher_rate))

            for i in range(num_years):
                row = {
                    "product_type": "Whole Life products",
                    "provider": provider,
                    "product": product,
                    "Subtitle": subtitle,
                    "Age": self.search_data["age"],
                    "Gender": self.search_data["gender"],
                    "Smoker": self.search_data["smoker"],
                    "Critical Illness benefit": self.search_data["critical_illness"],
                    "Premium Type": "Annual",
                    "Sum Assured": self.search_data["sum_assured"],
                    "Coverage Term": coverage_term,
                    "Premium Term": premium_term,
                    "Total Premium": tot_premium,
                    "Main Plan Annual Premium": main_plan_annual_premium,
                    "DB_Year": db_years[i],
                    "DB_Guaranteed": db_guaranteed[i],
                    "DB_Lower Rate": db_lower_rate[i],
                    "DB_Higher Rate": db_higher_rate[i],
                    "SV_Year": sv_years[i],
                    "SV_Guaranteed": sv_guaranteed[i],
                    "SV_Lower Rate": sv_lower_rate[i],
                    "SV_Higher Rate": sv_higher_rate[i],
                }
                product_rows.append(row)
            # Print the rows
                        
            data.extend(product_rows)
            self.save_to_excel()

            # go back
            driver.back()
            time.sleep(1)

            index += 1
            if index >= total_products:
                break

        return data
    
    def save_to_excel(self):
        global data

        # print("innerHTML data:", data)
        df = pd.DataFrame(data)
        file_name = "Whole_Life_Products.xlsx"
        try:
            df.to_excel(file_name, index=False)
            print(f"Excel file created: {file_name}")
        except PermissionError:
            # Handle the case where the file is open or locked
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            backup_file_name = f"Whole_Life_Products_{timestamp}.xlsx"
            df.to_excel(backup_file_name, index=False)
            print(f"Original file is locked. Saved as: {backup_file_name}")

    def js_click(self, element):
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
    
    def parse_currency(self, text):
        cleaned = re.sub(r"[^\d.]", "", text)
        return float(cleaned) if cleaned else 0.0

    def wait_element(self, using, css_code, timeout, wait_all=False):
        if wait_all:
            WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((using, css_code)))
            elements = driver.find_elements(using, css_code)
            return elements
        else:
            try:
                element = WebDriverWait(driver, timeout).until(
                    EC.presence_of_element_located((using, css_code))
                )
                return element
            except Exception as e:
                return None

product_type="Whole Life products"


term_life_data=get_whole_data()


count = 0
while count < len(term_life_data):
    data_combination = term_life_data[count]
    print(data_combination)
    bot = WholeifeAutomation(search_data=data_combination)
    bot.do_search()
    output = bot.scrape_products()

    if output is not None:
        print("-------------------------------------------------------------------------")
        print("increasing count to +1 coz in last iteration there was gateway problem. ")
        print("-------------------------------------------------------------------------")
        count += 1
                

16
{'age': 20, 'smoker': 'N', 'gender': 'M', 'premium_term': '11 to 15', 'sum_assured': 'S$ 200,000', 'critical_illness': 'N', 'category': 'non-dpi'}
Setting search options for Whole Life products
coverage term text:  {'age': 20, 'smoker': 'N', 'gender': 'M', 'premium_term': '11 to 15', 'sum_assured': 'S$ 200,000', 'critical_illness': 'N', 'category': 'non-dpi'}
Searching is done...
Total products text:  (164)
scraping data for product 1 of 164
Finished scrolling after 10 attempts
checking provider:  AIA Singapore
checking product:  AIA Guaranteed Protect Plus (IV)
Skipping product 1 - Provider: AIA Singapore, Product: AIA Guaranteed Protect Plus (IV)
scraping data for product 2 of 164
Finished scrolling after 10 attempts
checking provider:  AIA Singapore
checking product:  AIA Guaranteed Protect Plus (IV)
Skipping product 2 - Provider: AIA Singapore, Product: AIA Guaranteed Protect Plus (IV)
scraping data for product 3 of 164
Finished scrolling after 10 attempts
checking provider:  AI